# 한국 지역내총생산(GRDP)의 증가율과 고용률의 관계에 대한 가설 검정
경제 성장이 고용 창출을 이루어 내는지에 대한 연구를 통하여 관련성을 입증하고, 특히 한국의 각 지역별(광역시, 특별시, 도)의 총생산과 고용률을 데이터로 사용하여 연구함으로써 수도권과 지방의 불균형으로 인해 지방 소멸이 큰 문제로 대두되고 있는 한국에서 각 지역의 경제적 특성과 요구에 맞춘 정책 개발과 실행을 가능케 하며, 지역 간 불균형 해소, 사회적 안정, 자원 활용, 경제 경쟁력 강화 등에 도움을 줄 수 있다.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox, shapiro
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

df1 = pd.read_excel("grdp.xlsx", index_col=0) # use first column as index
df2 = pd.read_csv("employment.csv", encoding='cp949', index_col=0) 

In [3]:
# 변수1 (GRDP 성장률) 데이터 전처리
#df1 = df1.T
print(df1.head())
#df1.astype(float).plot()

    1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  ...  2012   
전국  10.8  12.3  12.1   6.9  10.3  10.6   5.8   6.8   9.1  10.1  ...   2.3  \
서울  10.8  13.8  10.6   7.6  10.3   8.8   8.2   8.1   6.4   6.8  ...   1.4   
부산   9.1  10.4   9.5   1.4  14.7   6.4   0.3   5.2   6.7   9.4  ...   2.6   
대구  10.8  12.9   7.7   4.2  13.6   3.3   1.5   3.8   8.7   8.0  ...   3.7   
인천   9.8  13.5  15.1   7.3  15.8  18.1   1.8   6.1   6.2  16.3  ...   3.7   

    2013  2014  2015  2016  2017  2018  2019  2020  2021  
전국   3.1   3.1   2.8   2.9   3.1   2.9   2.2  -0.6   4.2  
서울   1.6   1.5   2.9   2.8   2.3   3.6   2.6   0.9   3.5  
부산   1.7   4.2   3.0   1.7   1.6   1.7   2.5  -3.5   2.3  
대구   3.5   2.6   3.0   0.0   1.7   2.4   1.5  -2.0   4.4  
인천   2.7   4.8   1.8   2.6   4.3   0.7   1.8  -3.5   6.0  

[5 rows x 36 columns]


In [4]:
# 변수2 (고용률) 데이터 전처리
#df2 = df2.T
print(df2.head())
#df2.astype(float).plot()

       2000  2001  2002  2003  2004  2005  2006  2007  2008  2009  ...  2013   
계      61.5  62.2  63.4  63.0  63.8  63.7  64.0  64.1  64.0  63.0  ...  64.6  \
서울특별시  60.9  61.5  62.4  62.6  63.8  64.3  64.4  64.7  64.3  62.7  ...  65.0   
부산광역시  58.4  59.3  61.7  59.2  59.6  60.1  60.5  60.7  60.3  59.3  ...  61.8   
대구광역시  59.2  60.3  62.0  61.0  62.2  61.9  61.8  61.9  61.3  61.1  ...  63.2   
인천광역시  61.5  62.6  63.8  62.7  62.7  62.8  63.4  64.0  63.2  63.2  ...  65.3   

       2014  2015  2016  2017  2018  2019  2020  2021  2022  
계      65.6  65.9  66.1  66.6  66.6  66.8  65.9  66.5  68.5  
서울특별시  65.7  65.5  65.8  66.4  66.0  66.3  65.9  66.6  68.4  
부산광역시  62.7  62.7  62.6  63.4  62.9  64.2  62.9  63.9  65.4  
대구광역시  64.3  65.4  65.3  65.0  64.2  64.2  63.0  65.5  66.9  
인천광역시  65.4  65.9  67.2  67.3  68.6  68.3  66.9  67.4  69.4  

[5 rows x 23 columns]


## 귀무가설 (H0)
지역내경제성장률의 증가와 고용률의 증가는 관계가 없다. (등락을 같이 하지 않는다)
## 대립가설 (Ha)
지역내경제성장률이 증가하니 고용률도 증가했다. (등락을 같이 한다)

아마 growth rate가 높을 때와 그렇지 않을 때의 고용률의 추이를 보면 되지 않을까?
- 그러면 데이터 전처리를 GRDP 증가가 높을때의 고용률과 그렇지 않을 때의 고용률로 나누고하면 될듯?
- 성장률 평균을 계산하여 평균보다 높을때와 낮을때의 케이스들을 보면 된다

증가율로 하지 말고 GRDP 자체로 하는것도 좋을 듯
- 뭐 서울은 두 변수 사이의 상관관계가 있는데 부산은 관계가 없다 같은걸로 해도 될듯

In [5]:
# 먼저 서울특별시의 GRDP성장률과 고용률만 비교를 해보자.
'''
서울의 평균 GRDP 성장률은 5.57%
평균 성장률이 5.57%와 같거나 높은 년도와 낮은 년도의 고용률을 나누기 위해서 데이터 전처리를 한다.
둘의 비교
'''
#df1 = df1.T
#df2 = df2.T
seoul1 = df1.iloc[1:2,14:]
seoul2 = df2.iloc[1:2,:22]
seoul1 = seoul1.T
seoul2 = seoul2.T
print(seoul1)
print(seoul2)

        서울
2000  10.0
2001   4.3
2002   6.9
2003   1.8
2004   1.2
2005   4.3
2006   4.2
2007   5.9
2008   2.5
2009   1.5
2010   3.5
2011   3.2
2012   1.4
2013   1.6
2014   1.5
2015   2.9
2016   2.8
2017   2.3
2018   3.6
2019   2.6
2020   0.9
2021   3.5
      서울특별시
2000   60.9
2001   61.5
2002   62.4
2003   62.6
2004   63.8
2005   64.3
2006   64.4
2007   64.7
2008   64.3
2009   62.7
2010   63.6
2011   64.7
2012   64.7
2013   65.0
2014   65.7
2015   65.5
2016   65.8
2017   66.4
2018   66.0
2019   66.3
2020   65.9
2021   66.6


In [6]:
seoul1 = sm.add_constant(seoul1)

model = sm.OLS(seoul2, seoul1)
result = model.fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  서울특별시   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     7.148
Date:                Tue, 23 May 2023   Prob (F-statistic):             0.0146
Time:                        19:06:39   Log-Likelihood:                -37.837
No. Observations:                  22   AIC:                             79.67
Df Residuals:                      20   BIC:                             81.86
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         65.7210      0.565    116.378      0.0

In [ ]:
# 